# Uniform estimates for E1 and E2

This document is based on the [derivation](http://michaelnielsen.org/polymath1/index.php?title=Controlling_H_t-A-B/B_0#Estimation_of_E_1.2CE_2) of estimates of $E1/B^{eff}_0$ and $E2/B^{eff}_0$ which are uniform on prescribed intervals, $\Re{z} \in [x_N, x_{N+1})$, where $x_N := 4 \pi N^2 - \frac{\pi t}{4}$, $t=0.4$, $z=x+iy$, $x \ge 100$ and where $y$ is constant and $y \ge 1/3$. The following convention holds throughout, 

$$s = \frac{1+iz}{2} = \frac{1-y}{2} + i\frac{x}{2} = \sigma + iT,$$

where $\frac{x_N}{2} \le T < \frac{x_{N+1}}{2}.$ 

Equation 2.1 of the derivation is

$$|E_2/B^{eff}_0| = \frac{1}{T-3.33} \epsilon'(\frac{1+y+ix}{2})$$

which is shown to be majorized as follows.

$$\epsilon'(\frac{1+y+ix}{2}) \leq \frac{1}{2} \sum_{n=1}^N \frac{1}{n^{\frac{1+y}{2} + \frac{t}{2} \Re( \alpha_1(s^+_N)) - \frac{t}{2} \kappa - \frac{t}{4} \log n}} e^{ \frac{1}{2(T - 3.33)} c^+_n } c^+_n$$

with corresponding code 
* a constant: `b = (1+y)/2 + t/2*real(αeff(s⁺N))-t/2*κ`
* summand denominator: `n^(b-t/4*log(n))`

where

$x_N := 4 \pi N^2 - \frac{\pi t}{4}$, and is coded as a function `x(n) = 4*π*n^2 - π*t/4`

$s^+_N := \frac{1+y+i(x_N+x_{N+1})/2}{2},\:$  is coded as `s⁺N = (1+y+im*(x(N)+x(N+1))/2)/2`

$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi},\:$ (was previously implemented as αeff)

$\kappa := \frac{x_{N+1}-x_N}{4 (x_N-6)},\:$ is coded as `κ = (x(N+1)-x(N))/(4*(x(N)-6))`

$c^+_n := \frac{t^2}{4} (|\alpha_1(s^+_N) - \log n|^2 + 2 \kappa |\alpha_1(s^+_N) - \log n| + \kappa^2) + \frac{1}{3} + t,$

with corresponding code
* `a(n) = abs(αeff(s⁺N)-log(n))`, a function of n
* `c⁺(n) = t^2/4*(a(n)+κ)^2+1/3+t`, also a function of n


The code equivalents of expressions above are combined in the following function. Note that expressions such as `x(n)` are internally defined functions written in Julia "shorthand."

In [1]:
using DBNUpperBound
using DBNUpperBound.Asymptotics
αeff = DBNUpperBound.Asymptotics.αeff # not exported

""" E2_over_B0eff()
    
    Returns a uniform upper bound for E2/B0eff on the interval xN(N) ≤ ℜ(z) < xN(N+1).
    """
function E2_over_B0eff(N::Int, t::Real, y::Real)
    y ≥ 1/3 || error("Bound applies only if y ≥ 1/3; y = ",y," is out of range.")
    x(n) = 4*π*n^2 - π*t/4 # note: a function
    s⁺N = (1+y+im*(x(N)+x(N+1))/2)/2
    κ = (x(N+1)-x(N))/(4*(x(N)-6))
    a(n) = abs(αeff(s⁺N)-log(n))
    c⁺(n) = t^2/4*(a(n)+κ)^2+1/3+t
    b = (1+y)/2 + t/2*real(αeff(s⁺N))-t/2*κ # a constant for the following sum
    psum = 0.0
    # Note the following expressions in T are maximized when T=x/2 is minimized hence
    T = x(N)/2
    for n in 1:N
        c = c⁺(n)
        psum += c*exp(c/(2*(T-3.33)))/n^(b-t/4*log(n)) 
    end
    return 1/(T-3.33)*psum/2
end

E2_over_B0eff

A quick check with results from Python function [`habbbound`](https://github.com/km-git-acc/dbn_upper_bound/blob/fc2eda232b5904f6938301a044c9c52177b1e197/dbn_upper_bound/python/research/habb_bound.py#L20) follows.

In [2]:
@show E2_over_B0eff(3, .4, .4) # compare with 0.0152441562686686948196347530247 (Python habbbound(3))
@show E2_over_B0eff(4, .4, .4) # compare with 0.00950458681293757320628891207112 (Python habbbound(4))
nothing

E2_over_B0eff(3, 0.4, 0.4) = 0.015244156268668695
E2_over_B0eff(4, 0.4, 0.4) = 0.009504586812937572


Equation 2.2 of the derivation is,

$$|E_2/B^{eff}_0| = \frac{1}{T-3.33} \lambda \epsilon'(\frac{1-y+ix}{2})$$

which is shown to be majorized as follows:

$$\epsilon'(\frac{1-y+ix}{2}) \leq \frac{1}{2} \sum_{n=1}^N \frac{1}{n^{\frac{1-y}{2} + \frac{t}{2} \mathrm{Re} \alpha_1(s^-_N) - \frac{t}{2} \kappa - \frac{t}{4} \log n}} \exp( \frac{1}{2(T - 3.33)} c^-_n ) c^-_n$$

with corresponding code 
* a constant: `b = (1-y)/2 + t/2*real(αeff(s⁻N))-t/2*κ`
* summand denominator: `n^(b-t/4*log(n))`

where, as above,

$x_N := 4 \pi N^2 - \frac{\pi t}{4}$, and is coded as a function `x(n) = 4*π*n^2 - π*t/4`

$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi},\:$ (was previously implemented as αeff)

$\kappa := \frac{x_{N+1}-x_N}{4 (x_N-6)},\:$ is coded as `κ = (x(N+1)-x(N))/(4*(x(N)-6))`

and where,

$s^-_N := \frac{1-y+i(x_N + x_{N+1})}{2},\:$  is coded as `s⁻N = (1-y+im*(x(N)+x(N+1))/2)/2`

$c^-_n := \frac{t^2}{4} (|\alpha_1(s^-_N) - \log n|^2 + 2 \kappa |\alpha_1(s^-_N) - \log n| + \kappa^2) + \frac{1}{3} + t,$

with corresponding code
* `a(n) = abs(αeff(s⁻N)-log(n))`, a function of n
* `c⁻(n) = t^2/4*(a(n)+κ)^2+1/3+t`, also a function of n

and

$$\lambda \leq e^\delta N^{-y}$$

where

$\delta := \frac{\pi y}{2(x_N-6 - \frac{14+2y}{\pi})} + \frac{2y(7+y)}{x_N^2} \log \frac{|1+y+i x_{N+1}|}{4\pi}$

which is coded as `δ = π*y/(2(x(N)-6-(14+2*y)/π)) + 2*y*(7+y)/x(N)^2*log(abs(1+y+im*x(N+1))/(4*π))`

In [3]:

function E1_over_B0eff(N::Int, t::Real, y::Real)
    y ≥ 1/3 || error("Bound applies only if y ≥ 1/3; y = ",y," is out of range.")
    x(n) = 4*π*n^2 - π*t/4 # note: a function
    s⁻N = (1-y+im*(x(N)+x(N+1))/2)/2
    κ = (x(N+1)-x(N))/(4*(x(N)-6))
    a(n) = abs(αeff(s⁻N)-log(n))
    c⁻(n) = t^2/4*(a(n)+κ)^2+1/3+t
    δ = π*y/(2(x(N)-6-(14+2*y)/π)) + 2*y*(7+y)/x(N)^2*log(abs(1+y+im*x(N+1))/(4*π))
    b = (1-y)/2 + t/2*real(αeff(s⁻N))-t/2*κ # a constant for the following sum
    psum = 0.0
    # Note the following expressions in T are maximized when T=x/2 is minimized hence
    T = x(N)/2
    for n in 1:N
        c = c⁻(n)
        psum += c*exp(c/(2*(T-3.33)))/n^(b-t/4*log(n)) 
    end
    return 1/(T-3.33)*exp(δ)*N^(-y)*psum/2
end

E1_over_B0eff (generic function with 1 method)

A quick check with results from Python function [`habbbound`](https://github.com/km-git-acc/dbn_upper_bound/blob/fc2eda232b5904f6938301a044c9c52177b1e197/dbn_upper_bound/python/research/habb_bound.py#L20) follows.

In [4]:
@show E1_over_B0eff(3, .4, .4) # compare with 0.0118525967548234584001318287497 (Python habbbound(3))
@show E1_over_B0eff(4, .4, .4) # compare with 0.0069147234260213191684489866477 (Python habbbound(4))
nothing

E1_over_B0eff(3, 0.4, 0.4) = 0.01185259675482346
E1_over_B0eff(4, 0.4, 0.4) = 0.0069147234260213165
